In [1]:
import pickle

import sqlalchemy as sa
import pandas as pd
import data.constant as con
from tqdm import tqdm
import component.create_raceID as cr
import classes.db_operation_class as db
import classes.convert_df as convert
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import component.create_raceID as cr
from tqdm import tqdm
import pickle
import IPython.core.display as display
import IPython.display
from sklearn.linear_model import LinearRegression
# 評価指標：決定係数
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
# インスタンスの作成
db_instans = db.Main()
convert_instans = convert.Main()
raceIdList = cr.get_save_race_id()

In [3]:
columns_name = ['DISTANCE', 'WEIGHT', 'HORSE_TOTAL', 'PRIZE', 'HORSE_WEIGHT',
       'WEIGHT_GAIN_LOSS', 'GENDER_セ', 'GENDER_牝', 'GENDER_牡', 'GRADE_オープン',
       'GRADE_新馬', 'GRADE_未勝利', 'GRADE_１勝クラス', 'GRADE_２勝クラス', 'GRADE_３勝クラス',
       'PLACE_中京', 'PLACE_中山', 'PLACE_京都', 'PLACE_函館', 'PLACE_小倉', 'PLACE_新潟',
       'PLACE_札幌', 'PLACE_東京', 'PLACE_福島', 'PLACE_阪神', 'WEATHER_小雨',
       'WEATHER_小雪', 'WEATHER_晴', 'WEATHER_曇', 'WEATHER_雨', 'WEATHER_雪',
       'GROUND_ダート', 'GROUND_芝', 'GROUND_障害', 'SPIN_右', 'SPIN_左', 'SPIN_直',
       'SPIN_障害', 'SITUATION_不良', 'SITUATION_稍重', 'SITUATION_良',
       'SITUATION_重']

predDf = pd.DataFrame([],columns = columns_name)


In [4]:
# race_id = "201606050901"
race_id = "202303030604"



df = convert_instans.sql(f"""
SELECT *
FROM result_horse
LEFT JOIN race
    ON result_horse.race_id = race.race_id
        LEFT JOIN prize_money
        ON race.race_id = prize_money.race_id
        AND prize_money.ranking = 1
WHERE result_horse.race_id = "{race_id}";
                        """)

# 走破タイム予測用カラム
columns = ["RUN_TIME","DISTANCE","WEIGHT","HORSE_TOTAL","PRIZE","HORSE_WEIGHT","WEIGHT_GAIN_LOSS","GENDER","GRADE","PLACE","WEATHER","GROUND","SPIN","SITUATION"]
runtimeDf = df[columns]

df

,RACE_ID,RANKING,HORSEFRAME,HORSENUMBER,HNAME,HORSE_ID,GENDER,AGE,WEIGHT,JOCKEY,...,TIME,RACEDATE,PLACE,GRADE,LIMIT,HANDICAP,RACE_TYPE,RACE_ID,RANKING,PRIZE
0,202303030604,1,7,12,エイムトゥルー,2018105575,牡,5,60.0,石神,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
1,202303030604,2,4,6,キールロワイヤル,2020102586,牡,3,58.0,平沢,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
2,202303030604,3,5,7,ウラエウス,2018104787,セ,5,60.0,中村,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
3,202303030604,4,5,8,エルデスペラード,2019100953,牡,4,60.0,田村,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
4,202303030604,5,3,3,ビップランバン,2018105050,牡,5,60.0,難波,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
5,202303030604,6,6,9,ホウオウフラッシュ,2019105826,牝,4,58.0,江田勇,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
6,202303030604,7,6,10,アルカンサス,2019101023,牡,4,60.0,蓑島,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
7,202303030604,8,8,13,ピーターサイト,2020105611,牡,3,58.0,上野,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
8,202303030604,9,4,5,ファーストリッキー,2019101107,セ,4,60.0,小牧加,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790
9,202303030604,10,8,14,クリノエイブラハム,2020101399,牡,3,58.0,大庭,...,11:20,2023年11月19日,福島,未勝利,無,定量,,202303030604,1,790


In [5]:
predinfoDf = pd.get_dummies(runtimeDf) # one-hot-encofing
predinfoDf = predinfoDf * 1              # true flaseを1 0に変換
predinfoDf = predinfoDf[predinfoDf["RUN_TIME"] > 0] #走破タイムが0以下の行を削除
predinfoDf = predinfoDf.iloc[:,1:]
predinfoDf


,DISTANCE,WEIGHT,HORSE_TOTAL,PRIZE,HORSE_WEIGHT,WEIGHT_GAIN_LOSS,GENDER_セ,GENDER_牝,GENDER_牡,GRADE_未勝利,PLACE_福島,WEATHER_晴,GROUND_障害,SPIN_障害,SITUATION_良
0,2770,60.0,14,790,514,-2,0,0,1,1,1,1,1,1,1
1,2770,58.0,14,790,462,-6,0,0,1,1,1,1,1,1,1
2,2770,60.0,14,790,456,-4,1,0,0,1,1,1,1,1,1
3,2770,60.0,14,790,458,2,0,0,1,1,1,1,1,1,1
4,2770,60.0,14,790,470,-4,0,0,1,1,1,1,1,1,1
5,2770,58.0,14,790,458,4,0,1,0,1,1,1,1,1,1
6,2770,60.0,14,790,468,-6,0,0,1,1,1,1,1,1,1
7,2770,58.0,14,790,468,4,0,0,1,1,1,1,1,1,1
8,2770,60.0,14,790,486,-2,1,0,0,1,1,1,1,1,1
9,2770,58.0,14,790,502,-10,0,0,1,1,1,1,1,1,1


In [6]:
# 重複しないカラムに初期値0を挿入
predinfoDf_columuns = set(predinfoDf.columns)
pred_columns = set(predDf.columns)
add_columuns = pred_columns - predinfoDf_columuns
add_columuns = list(add_columuns)
for column in add_columuns:
  predinfoDf[column] = 0

In [7]:
# DataFrame2をDataFrame1のカラム順に並び替える
predinfoDf = predinfoDf.reindex(columns=predDf.columns)


In [12]:
# モデルの読み込み
with open("data/RUN_TIME_PREDICT.model", mode="rb") as f:
  model = pickle.load(f)

predtime = model.predict(predinfoDf)
print('予測のタイム')
print(predtime)
print('実際のタイム')
(df["RUN_TIME"])

予測のタイム
[190.08341633 190.2186445  190.210133   190.12010519 190.11312336
 190.34959599 190.11480037 190.21292898 190.18976931 190.19263374
 190.35260776]
実際のタイム


0     187.0
1     187.5
2     187.6
3     188.6
4     188.7
5     191.0
6     191.5
7     191.5
8     191.6
9     195.3
10    195.4
11      0.0
12      0.0
13      0.0
Name: RUN_TIME, dtype: float64